In [ ]:
__nbid__ = '0071'
__author__  = 'Brian Merino <brian.merino@noirlab.edu>, Vinicius Placco <vinicius.placco@noirlab.edu>, David Herrera <david.herrera@noirlab.edu>'
__version__ = '20251205' # aaaammdd; marca de fecha de la versión de este notebook
__keywords__ = ['GHOST','Gemini','stars','DRAGONS']

# Reducción de XX Oph Gemini con GHOST usando el API Python de DRAGONS
#### (_Traducción de [GHOST_IFU_Star.ipynb](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/DataReduction/DRAGONS_reduction_examples/GHOST_IFU_Star/GHOST_IFU_Star.ipynb)_)
***
## Datos públicos de archivo de ghost_tutorial - GS-ENG-GHOST-COM-3-915 (XX Oph)
#### adaptado de https://dragons.readthedocs.io/projects/ghost-drtutorial/en/stable/
***

### Nota: Este notebook puede tardar más de una hora en ejecutarse. Puede llevar más o menos tiempo si trabajas con un conjunto de datos diferente al utilizado en este tutorial. El tiempo total de ejecución depende de la cantidad y variedad de archivos que desee reducir.

### Si desea ejecutar este notebook en su máquina local, asegúrese de que su base de datos de calibración DRAGONS esté configurada correctamente. Para hacerlo, asegúrese de que la sección de calibraciones de ./dragons/dragonsrc se vea así:

```
[calibs]
databases = ~/.dragons/dragons.db get store
```

## Tabla de contenido
* [Objetivos](#goals)
* [Resumen](#summary)
* [Avisos legales y atribuciones](#disclaimer)
* [Importaciones y configuración](#imports)
* [Acerca del conjunto de datos](#About)
* [Preparar el directorio de trabajo](#Prepare)
* [Descarga de datos para reducción](#Downloading_Data)
* [Crear listas de archivos](#File_Lists)
* [Configurar el registrador DRAGONS](#DRAGONS_logger)
* [Crear update_list() y reducir_func()](#func)
* [Reducción de bias](#biases)
* [Reducción de flats](#flats)
* [Reducción de arcos](#arcs)
* [Reducción estándar espectroscópica](#spec_standard)
* [Reducción de la ciencia](#science)
* [Graficar espectros reducidos](#Plot)
* [Espectros 1D de salida](#write1DSpectra)
* [Guardar gráficos de espectros reducidos](#Save_plot)
* [Hacer compatible los espectros reducidos IRAF](#IRAF_compatible)

<a class="anchor" id="goals"></a>
# Objetivos
Mostrar cómo reducir los datos de espectroscopía GHOST usando el paquete Gemini DRAGONS en la plataforma científica Data Lab usando un kernel especial de DRAGONS `"DRAGONS (Py4.0)"`. Los pasos incluyen descargar datos del archivo Gemini, configurar el servicio de calibración DRAGONS, procesar bias, flats y arcos, crear flats maestros y flats de rendija, reducir las estándares y los datos científicos y, finalmente, crear los espectros reducidos finales para los brazos rojo y azul de GHOST.


<a class="anchor" id="summary"></a>
# Resumen
DRAGONS es una plataforma de reducción de datos astronómicos basada en Python escrita por el Departamento de soporte al usuario de Gemini Science. Actualmente se puede utilizar para reducir datos de imágenes de los instrumentos Gemini GMOS, NIRI, Flamingos 2, GSAOI y GNIRS, así como datos espectroscópicos tomados con GHOST y GMOS en modo de rendija larga. Enlazado [aquí](https://dragons.readthedocs.io/en/v4.0.0/) hay una lista general de guías, manuales y tutoriales sobre el uso de DRAGONS.

El kernel DRAGONS está disponible en el entorno Data Lab, lo que permite a los usuarios acceder a las rutinas sin depender de la instalación del software en sus máquinas locales. Es importante tener en cuenta que cuando se ejecuta un comando DRAGONS, el resultado se mostrará dentro de la celda. Asegúrese de desplazarse por el resultado para asegurarse de que no se pierdan errores.

Presentamos un ejemplo de un notebook Jupyter de DRAGONS que funciona en el entorno de Data Lab para reducir completamente los datos de espectroscopia Gemini South GHOST azul:2x2 y rojo:2x2. Este notebook no presentará todos los detalles de las muchas opciones disponibles para ajustar u optimizar el proceso de reducción de datos de DRAGONS GHOST; más bien, solo mostrará un ejemplo de una reducción estándar de un conjunto de datos espectroscópicos GHOST.

Los datos utilizados en este ejemplo de notebook son datos de espectroscopía GHOST azul:2x2 y rojo:2x2 del archivo Gemini de la estrella XX Oph de la corrida para la puesta en servicio de GHOST. Debido a que los datos utilizados provienen de dicha corrida, no hay información del programa disponible, pero puede encontrar más información sobre las instrucciones de uso rojas y azules de GHOST en la [página del instrumento GHOST](https://www.gemini.edu/instrumentation/ghost).

Se descargarán y crearán varios archivos mientras se ejecuta este notebook. Asegúrese de tener al menos 3 GB de espacio de almacenamiento antes de ejecutar esta computadora portátil, de modo que haya suficiente espacio disponible para acomodar estos archivos.



<a class="anchor" id="attribution"></a>
# Avisos Legales y atribuciones

Avisos Legales
-----------
Tome en cuenta que usar el Astro Data Lab constituye un acuerdo con nuestros [Avisos Legales](https://datalab.noirlab.edu/disclaimers.php) mínimos.

Reconocimientos
---------------
Si ud. usa el **Astro Data Lab** en sus publicaciones de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios de datos proveeidos por el Astro Data Lab, el cual es parte del Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) del NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc."(Asociación de Universidaddes para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

Si utiliza **SPARCL junto con la plataforma de Astro Data Lab** (por medio de JupyterLab, línea de comando o interfaz de la web) en su publicación de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios o datos proporcionados por el "SPectra Analysis and Retrievable Catalog Lab" (SPARCL) (Laboratorio de Análisis y Catálogo Recuperable de Espectros) y el Astro Data Lab, ambos pertenecientes al Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) de NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc." (Asociación de Universidades para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

En cualquiera de los casos, **por favor cite las siguientes publicaciones**:

* Publicación del concepto de Data Lab: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411.

Si hace referencia al Jupyterlab / Jupyter notebooks de Data Lab, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097.

Si publica en una revista de la AAS, agregue también la palabra clave `\facility{Astro Data Lab}`

Y si está usando SPARCL, por vor agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576.

La biblioteca de NOIRLab mantiene [listas de reconocimientos apropiados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar cuando se hacen publicaciones utilizando los recursos, servicios o datos del Laboratorio.

Para este notebook en particular, por favor también incluya el siguiente reconocimiento:

* Publicación de **DRAGONS**: Labrie et al., <a href="https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract">"DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South"</a>, ASPC, 523, 321L

* <a href="https://zenodo.org/record/7776065#.ZDg5qOzMLUI">Publicación del software **DRAGONS** de código abierto</a>

---- **Versión en Inglés** ----


# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

For this notebook specifically, please acknowledge:
* DRAGONS publication: Labrie et al., [DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South](https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract), ASPC, 523, 321L 

* [DRAGONS open source software publication](https://zenodo.org/record/7776065#.ZDg5qOzMLUI)


<a class="anchor" id="imports"></a>
# Importaciones y configuración

In [ ]:
import warnings
import glob
import os
import numpy as np
import astrodata
import shutil
import matplotlib.pyplot as plt

from gempy.utils import logutils
from gempy.adlibrary import dataselect

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.filterwarnings("ignore")

## Establezca una función para verificar y eliminar varios directorios o archivos.

In [ ]:
def check_and_delete(path):
    # Verifique si la ruta ya existe.
    if os.path.exists(path):
        # Si la ruta conduce a un directorio.
        if os.path.isdir(path):
            # Eliminar directorio
            shutil.rmtree(path)
            
        # Si la ruta conduce a un archivo.
        elif os.path.isfile(path):
            # Eliminar archivo
            os.remove(path)

<a class="anchor" id="About"></a>
# Acerca del conjunto de datos

Los datos de GHOST utilizados para este tutorial son de la estrella XX Oph. Se utilizó IFU-1 para obtener una resolución estándar de la estrella. Los datos se obtuvieron durante la corrida de puesta en servicio.

La siguiente tabla contiene un resumen del conjunto de datos:

| Tipo de observación | Nombre(s) de archivo | IFU, agrupación y modo de lectura |
| :--- | :--- | :---: |
| Ciencia | S20230416S0079 | azul: 2x2, lento; rojo:2x2, medio |
| Bias de ciencia | S20230417S0011-015 |  |
| Flats de ciencia | S20230416S0047 | 1x1; azul: lento; rojo:medio |
| Arcos de ciencia | S20230416S0049-51 | 1x1; azul: lento; rojo:medio |
| Bias de flats <br> Bias de arco | S20230417S0036-40 | 1x1; azul: lento; rojo:medio |
| Estándar (CD-32 9927) | S20230416S0073 | azul: 2x2, lento; rojo:2x2, medio |
| Biases de estándar <br> Flats de estándar <br> Arcos de estándar <br> Biases de flats estándar <br> Biases de arco estándar | Utilice calibraciones de ciencia |
| BPM | bpm_20220601_ghost_blue_11_full_4amp.fits <br> bpm_20220601_ghost_red_11_full_4amp.fits |  |

<a class="anchor" id="Prepare"></a>
# Preparar el directorio de trabajo

Si tiene archivos intermedios que se crearon al ejecutar este código en el pasado, deberá eliminarlos de su directorio de trabajo. La siguiente celda define una función de limpieza que eliminará todos los archivos fits de su directorio de trabajo. Esta función se llamará nuevamente al final del tutorial, dejándolo solo con el producto final.

De forma predeterminada, esta función eliminará todos los archivos de registro, listas y fits en el directorio de trabajo. Si hay archivos que se han reducido previamente y que le gustaría conservar, configure `save_reduced=1` al llamar a la función.

In [ ]:
def clean_up(save_reduced=0):
    # ¿Ya existe el directorio de calibraciones?
    check_and_delete('calibrations')

    # Elimine los archivos de registro y lista existentes.
    work_dir_path = os.getcwd()
    work_dir = os.listdir(work_dir_path)

    for item in work_dir:
        if item.endswith(".log") or item.endswith(".list"):
            check_and_delete(os.path.join(work_dir_path,item))
    
    # A continuación, eliminaremos todos los archivos fits existentes, excepto los archivos previamente reducidos, dependiendo de cómo haya configurado save_reduced.
    if save_reduced:
        all_files = glob.glob('*.fits')
        save = dataselect.select_data(all_files, [], ['PROCESSED'])
        
        for s in save:
            check_and_delete(os.path.join(work_dir_path,s))

        if os.path.exists(os.path.join('reduced')):
            remain_files = os.listdir(work_dir_path)
        
            for item in remain_files:
                if os.path.splitext(item)[1] in ('.dat','.pdf','.fits','.png'):
                    # Compruebe si el archivo ya existe en formato reducido/
                    # Si es así, elimínelo y reemplácelo con la nueva copia.
                    if os.path.exists(os.path.join(work_dir_path,'reduced',item)):
                        check_and_delete(os.path.join(work_dir_path,'reduced',item))
                        shutil.move(item,os.path.join(work_dir_path,'reduced'))

                    else:
                        shutil.move(item,os.path.join(work_dir_path,'reduced'))
        
        # Crear el directorio reduced/ y mover archivos reducidos
        else:
            os.mkdir(os.path.join(work_dir_path,'reduced'))
            remain_files = os.listdir(work_dir_path)
        
            for item in remain_files:
                if os.path.splitext(item)[1] in ('.dat','.pdf','.fits','.png'):
                    shutil.move(item,os.path.join(work_dir_path,'reduced'))

    else:
        all_files = glob.glob('*.fits')
        for a in all_files:
            check_and_delete(os.path.join(work_dir_path,a))

## Crear un directorio para archivos sin procesar
Este tutorial creará una gran cantidad de archivos intermedios que se almacenarán temporalmente en el directorio de trabajo. Para garantizar que no se pierda ninguno de los datos originales, crearemos un directorio llamado raw para almacenar los datos preliminares de forma segura.

In [ ]:
check_and_delete('raw')
os.mkdir('raw')

<a class="anchor" id="Downloading_Data"></a>
# Descargando los datos

Descargue los datos espectroscópicos y de calibración del archivo Gemini al directorio de trabajo actual. Este paso sólo debe ejecutarse una vez.

Si ejecuta este notebook por primera vez y necesita descargar el conjunto de datos, configure la variable ""download = True". El notebook no volverá a descargar el conjunto de datos si está configurado en False. Esto resultará especialmente útil si ejecuta los notebooks más de una vez.

In [ ]:
%%bash 

# Cree una lista de archivos FITS para descargar.
echo "\
https://archive.gemini.edu/file/S20230416S0047.fits
https://archive.gemini.edu/file/S20230416S0049.fits
https://archive.gemini.edu/file/S20230416S0050.fits
https://archive.gemini.edu/file/S20230416S0051.fits
https://archive.gemini.edu/file/S20230416S0073.fits
https://archive.gemini.edu/file/S20230416S0079.fits
https://archive.gemini.edu/file/S20230417S0011.fits
https://archive.gemini.edu/file/S20230417S0012.fits
https://archive.gemini.edu/file/S20230417S0013.fits
https://archive.gemini.edu/file/S20230417S0014.fits
https://archive.gemini.edu/file/S20230417S0015.fits
https://archive.gemini.edu/file/S20230417S0036.fits
https://archive.gemini.edu/file/S20230417S0037.fits
https://archive.gemini.edu/file/S20230417S0038.fits
https://archive.gemini.edu/file/S20230417S0039.fits
https://archive.gemini.edu/file/S20230417S0040.fits
https://archive.gemini.edu/file/bpm_20220601_ghost_blue_11_full_4amp.fits
https://archive.gemini.edu/file/bpm_20220601_ghost_red_11_full_4amp.fits\
" > ghost.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -P './raw' -i ghost.list

else
    echo "Omitiendo la descarga. Para descargar el conjunto de datos utilizado en este notebook, establezca download=True."
fi

<a class="anchor" id="File_Lists"></a>
# Crear listas de archivos

Este conjunto de datos contiene imágenes científicas y de calibración. Para algunos programas, podría haber diferentes objetivos observados y tiempos de exposición dependiendo de cómo organice sus datos sin procesar. El proceso de reducción de datos de DRAGONS no organiza los datos por ti. Tú tienes que hacerlo. DRAGONS proporciona herramientas para ayudarte con eso.

El primer paso es crear listas que se utilizarán en el proceso de reducción de datos.

In [ ]:
all_files = glob.glob('raw/S2023*.fits')
all_files.append(glob.glob('raw/bpm*.fits')[0])
all_files.append(glob.glob('raw/bpm*.fits')[1])
all_files.sort()

<a class="anchor" id="DRAGONS_logger"></a>
# Configurando el registrador DRAGONS

DRAGONS viene con un administrador de calibraciones local que utiliza las mismas reglas de asociación de calibración que el Archivo del Observatorio Gemini. Esto permite que `reduce` realice solicitudes a una base de datos local liviana para hacer coincidir las calibraciones procesadas cuando sea necesario para reducir un conjunto de datos.

Esto le indica al sistema dónde colocar la base de datos de calibración. Esta base de datos realizará un seguimiento de las calibraciones procesadas que le enviaremos.

In [ ]:
logutils.config(file_name='ghost.log')
caldb = cal_service.set_local_database()
caldb.init("w")

## Agregue las máscaras de píxeles malos a la base de datos de calibración

In [ ]:
caldb.add_cal(glob.glob('raw/bpm*.fits')[0])
caldb.add_cal(glob.glob('raw/bpm*.fits')[1])

<a class="anchor" id="func"></a>
# update_list() y reduce_func()

Este notebook requerirá actualizar la lista de archivos en su directorio de trabajo y llamar al comando `reduce` varias veces. Para reducir el texto repetitivo, hemos creado dos funciones que reducirán el número de líneas incluidas en este notebook.

In [ ]:
def update_list():
    # Cree una nueva lista de archivos que contenga los archivos intermedios.
    # identificar todos los archivos en el directorio de trabajo
    intermediate = os.listdir()
    new_all_files = []

    # Dado que os.listdir() devuelve todos los archivos en el directorio de trabajo
    # este bucle seleccionará sólo los archivos adecuados y los agregará a una lista.
    for i in intermediate:
        if os.path.splitext(i)[1] == '.fits':
            new_all_files.append(i)
    
    print('%i archivos en la lista.'%len(new_all_files))
    return new_all_files

In [ ]:
def reduce_func(files_list,uparms=None,recipename=None):
    # Utilice la función 'reduce' de DRAGONS para reducir la lista de archivos proporcionada.
    # De forma predeterminada, esta función utilizará la configuración predeterminada para reduce().
    # uparms: esta es una lista de tuplas con el nombre primitivo y el parámetro en el primer elemento y el valor en el segundo, p.e. [('stackFrames:operation', 'median')].
    # nombre de receta: el nombre de la receta que se utilizará, p. 'makeIRAFCompatible'.
    reduce = Reduce()
    reduce.files.extend(files_list)

    if uparms is not None:
        reduce.uparms = [uparms]
    
    if recipename is not None:
        reduce.recipename = recipename
    
    reduce.runr()

<a class="anchor" id="biases"></a>
# Reducción de bias

Comience usando dataselect para identificar todos los archivos en all_files con la etiqueta 'BIAS'.

In [ ]:
biasbundles = dataselect.select_data(all_files, ['BIAS'], [])
print(biasbundles)

## Utilice reduce_func() para reducir los bias

Cuando esta celda termine de ejecutarse, se crearán tres archivos para cada bias (ciencia, flat y arco). Tendrán el sufijo *_blue001.fits, *_red001.fits y *_slit.fits.

In [ ]:
reduce_func(biasbundles)

## Actualizar lista

La función `reduce()` de DRAGONS crea una gran cantidad de archivos intermedios que se almacenan en el directorio de trabajo. Antes de volver a llamarlo, primero debemos actualizar nuestra lista de archivos usando `update_list()`.

In [ ]:
new_all_files = update_list()

## Ahora use dataselect para elegir los bias de rendija.

In [ ]:
biaslit = dataselect.select_data(new_all_files, ['BIAS','SLIT'])
print(biaslit)

## Reducir los bias de las rendijas.

Cuando termine de ejecutarse, se creará un nuevo archivo llamado S20230417S0011_slit_bias.fits.

In [ ]:
reduce_func(biaslit)

## Actualiza la lista de archivos.

In [ ]:
new_all_files2 = update_list()

## Utiliza la selección de datos para elegir los bias de ciencia rojos y reducirlos.

In [ ]:
expression = "binning==\'2x2\'"
parsed_expr = dataselect.expr_parser(expression)
biasredsci = dataselect.select_data(new_all_files2, ['BIAS', 'RED'], [], parsed_expr)

Una vez que termine de ejecutarse, existirá un nuevo archivo llamado S20230417S0011_red001_bias.fits en el directorio de trabajo.

In [ ]:
reduce_func(biasredsci)

## Selecciona los bias de ciencia azules y redúzcalos.

In [ ]:
expression = "binning=='2x2'"
parsed_expr = dataselect.expr_parser(expression)
biasbluesci = dataselect.select_data(new_all_files2, ['BIAS','BLUE'], [], parsed_expr)

Se creará un único archivo llamado S20230417S0011_blue001_bias.fits después de ejecutar esta celda.

In [ ]:
reduce_func(biasbluesci)

## Selecciona los bias de flats/arco rojos y redúzcalos.

In [ ]:
expression = "binning=='1x1'"
parsed_expr = dataselect.expr_parser(expression)
biasredflatarc = dataselect.select_data(new_all_files2, ['BIAS','RED'], [], parsed_expr)

Al ejecutar la siguiente celda se creará un nuevo archivo llamado S20230417S0036_red001_bias.fits.

In [ ]:
reduce_func(biasredflatarc)

## Selecciona los bias de flats/arcos azules y redúzcalos.

In [ ]:
expression = "binning=='1x1'"
parsed_expr = dataselect.expr_parser(expression)
biasblueflatarc = dataselect.select_data(new_all_files2, ['BIAS','BLUE'], [], parsed_expr)

Al ejecutar la siguiente celda se creará un nuevo archivo llamado S20230417S0036_blue001_bias.fits.

In [ ]:
reduce_func(biasblueflatarc)

## Limpieza

La reducción GHOST crea muchos archivos, a menudo grandes, en el directorio de trabajo. Se recomienda realizar una limpieza entre cada fase de reducción. Si desea guardar los archivos intermedios, muévalos (mv) a otro lugar. En este tutorial, simplemente los eliminaremos.

In [ ]:
%%bash

rm *fits

<a class="anchor" id="flats"></a>
# Reducción de flats
## Desagrupa los flats y luego redúcelos. Comenzando con los flats maestros y los flats de rendija.

In [ ]:
flatbundles = dataselect.select_data(all_files, ['FLAT'], [])

La ejecución de esta celda generará 11 archivos del flat de ciencia S20230416S0047.fits. Cinco tendrán el sufijo _blue00*.fits, cinco tendrán el sufijo _red00*.fits y el resto tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(flatbundles)

## Actualiza la lista de archivos.

In [ ]:
new_all_files3 = update_list()

## Selecciona los flats de rendija y redúcelos.

In [ ]:
slitflat = dataselect.select_data(new_all_files3, ['SLITFLAT'], [])

La siguiente celda creará un único archivo llamado S20230416S0047_slit_slitflat.fits.

Nota: Esta celda dará un ERROR con respecto a las entradas que tienen diferentes números de extensiones SCI. Este es un problema conocido con DRAGONS y aparecerá independientemente de los datos que proporciones. Este ERROR se puede ignorar sin problema.

In [ ]:
reduce_func(slitflat)

## Selecciona y reduce los flats rojos.

In [ ]:
flatred = dataselect.select_data(new_all_files3, ['FLAT','RED'], [])

Al ejecutar la siguiente celda también se creará un único archivo llamado S20230416S0047_red001_flat.fits.

In [ ]:
reduce_func(flatred)

## Selecciona y reduce los flats azules.

In [ ]:
flatblue = dataselect.select_data(new_all_files3, ['FLAT','BLUE'], [])

La siguiente celda creará un archivo llamado S20230416S0047_blue001_flat.fits.

In [ ]:
reduce_func(flatblue)

## Limpieza

In [ ]:
%%bash

rm *fits

<a class="anchor" id="arcs"></a>
# Arcos
## Desagrupa los arcos y redúcelos.

In [ ]:
arcbundles = dataselect.select_data(all_files, ['ARC'], [])

Al ejecutar la siguiente celda se crearán 9 archivos, tres para cada uno de los arcos científicos. Tres tendrán el sufijo _blue001.fits, tres tendrán el sufijo _red001.fits y los tres restantes tendrán el sufijo _slit.fits.

In [ ]:
reduce_func(arcbundles)

## Actualiza la lista de archivos.

In [ ]:
new_all_files4 = update_list()

## Selecciona y reduce las imágenes de rendija para el arco.

In [ ]:
arcslit_1 = dataselect.select_data(new_all_files4, ['ARC','SLIT'], [])

In [ ]:
# El tutorial original ordena sus listas numéricamente de forma predeterminada, mientras que esta versión no lo hace.
# Se han agregado algunas líneas de código aquí para ordenar manualmente la lista.
arcslit_1.sort()
arcslit_2 = [arcslit_1[0]]
print(arcslit_2)

La siguiente celda devolverá un único archivo llamado S20230416S0049_slit_slit.fits.

In [ ]:
reduce_func(arcslit_2)

## Selecciona y reduce los arcos rojos.

In [ ]:
arcred = dataselect.select_data(new_all_files4, ['ARC','RED'], [])
arcred.sort()
arcred

La siguiente celda creará un archivo llamado S20230416S0049_red001_arc.fits.

In [ ]:
reduce_func(arcred)

## Selecciona y reduce los arcos azules.

In [ ]:
arcblue = dataselect.select_data(new_all_files4, ['ARC','BLUE'], [])
arcblue.sort()
arcblue

La siguiente celda también creará un único archivo llamado S20230416S0049_blue001_arc.fits.

In [ ]:
reduce_func(arcblue)

## Limpieza

In [ ]:
%%bash

rm *fits

<a class="anchor" id="spec_standard"></a>
# Estándar espectroscópico
## Desagrupa las imágenes de estándares y redúcelas.

In [ ]:
expression = "object=='CD -32 9927'"
parsed_expr = dataselect.expr_parser(expression)
stdbundles = dataselect.select_data(all_files, [], [], parsed_expr)
stdbundles.sort()
stdbundles

La siguiente celda creará 5 archivos comenzando con el mismo nombre que el estándar. Uno tendrá el sufijo _blue001.fits, tres tendrán el sufijo _red00*.fits y el archivo final tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(stdbundles)

## Actualiza la lista de archivos.

In [ ]:
new_all_files5 = update_list()

## Selecciona los datos de la estándar del visor de rendija y redúcelos.

In [ ]:
stdslit = dataselect.select_data(new_all_files5, ['SLIT'], [])
stdslit.sort()
stdslit

A diferencia de las celdas anteriores, la siguiente celda creará cuatro nuevos archivos fits y un pdf. El pdf tendrá el sufijo _slit_slitflux. Un archivo fits tendrá el sufijo _slit_blue001_slit.fits y los tres restantes tendrán el sufijo _slit_red00*_slit.fits.

Nota: Esta celda dará un ERROR con respecto a las entradas que tienen diferentes números de extensiones SCI. Este es un problema conocido con DRAGONS y aparecerá independientemente de los datos que proporciones. Este ERROR se puede ignorar sin problemas.

In [ ]:
reduce_func(stdslit)

## Selecciona los datos rojos de la estrella estándar y redúcelos.

In [ ]:
stdred = dataselect.select_data(new_all_files5, ['RED'], [])
stdred.sort()
stdred

Al ejecutar la siguiente celda se generará un archivo llamado S20230416S0073_red001_standard.fits.

In [ ]:
reduce_func(stdred,uparms=('scaleCountsToReference:tolerance',1),recipename='reduceStandard')

## Selecciona los datos azules de la estrella estánda y redúcelos.

In [ ]:
stdblue = dataselect.select_data(new_all_files5, ['BLUE'], [])

In [ ]:
stdblue.sort()
stdblue

La siguiente celda también generará un único archivo llamado S20230416S0073_blue001_standard.fits.

In [ ]:
reduce_func(stdblue,uparms=('scaleCountsToReference:tolerance',1),recipename='reduceStandard')

<a class="anchor" id="clean_up_3"></a>
# Limpieza

In [ ]:
%%bash

rm *fits

<a class="anchor" id="science"></a>
# Ciencia
## Desagrupa las imágenes de ciencia y redúcelas.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
scibundles = dataselect.select_data(all_files, [], [],parsed_expr)

Al ejecutar la siguiente celda se crearán 5 archivos cuyos nombres comenzarán con el nombre del archivo científico. Uno tendrá el sufijo _blue001.fits, tres tendrán el sufijo _red00*.fits y el archivo final tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(scibundles)

## Actualiza la lista de archivos.

In [ ]:
new_all_files6 = update_list()

<a class="anchor" id="science_slit_viewer"></a>
# Selecciona los datos científicos del visor de rendijas y redúcelos.

In [ ]:
scislit  = dataselect.select_data(new_all_files6, ['SLIT'], [])

Al igual que cuando se reducen los datos científicos, la siguiente celda creará 5 archivos. Uno tendrá el sufijo _slit_blue001_slit.fits, tres tendrán el sufijo _slit_red00*_slit.fits y el último tendrá el sufijo _slit_slitflux.pdf.

Nota: Esta celda dará un ERROR con respecto a las entradas que tienen diferentes números de extensiones SCI. Este es un problema conocido con DRAGONS y aparecerá independientemente de los datos que proporciones. Este ERROR se puede ignorar con seguridad.

In [ ]:
reduce_func(scislit)

## Actualiza la lista de archivos.

In [ ]:
new_all_files7 = update_list()

## Selecciona y reduce las imágenes rojas de ciencia.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
scired = dataselect.select_data(new_all_files7, ['RED'], [])
scired = np.sort(scired)

Ejecutar la siguiente celda creará 4 archivos. Uno tendrá el sufijo _red001_dragons.fits y los otros tres tendrán el sufijo _red00*_calibrated.fits.

In [ ]:
reduce_func(scired)

## Selecciona y reduce las imágenes azules de ciencia.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
sciblue = dataselect.select_data(new_all_files7, ['BLUE'], [])
sciblue = np.sort(sciblue)

Ejecutar la siguiente celda creará dos archivos. El primero tendrá el sufijo _blue001_calibrated.fits y el otro tendrá el sufijo _blue001_dragons.fits.

In [ ]:
reduce_func(sciblue)

<a class="anchor" id="Plot"></a>
# Graficar los espectros rojo y azul reducidos

In [ ]:
# Desplegar S20230416S0079_red001_dragons.fits y S20230416S0079_blue001_dragons.fits
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(13,5))

red_file = 'S20230416S0079_red001_dragons.fits'
blue_file = 'S20230416S0079_blue001_dragons.fits'

red_pf  = astrodata.open(red_file)
blue_pf = astrodata.open(blue_file)

red_flux  = red_pf[0].data
blue_flux = blue_pf[0].data

red_flux_array  = np.array(red_flux)
blue_flux_array = np.array(blue_flux)

red_wave  = red_pf[0].wcs(np.arange(red_flux.size)).astype(np.float32)
blue_wave = blue_pf[0].wcs(np.arange(blue_flux.size)).astype(np.float32)

# Convertir λ de nm a Å
red_wave_array  = np.array(red_wave*10)
blue_wave_array = np.array(blue_wave*10)

ax1.plot(blue_wave_array,blue_flux_array,lw=0.4)
ax1.set_xlim(3450,5450)
ax1.set_ylim(-0.05*10**(-12),0.2*10**(-12))
ax1.set_xlabel('Longitud de onda [$\AA$]')
ax1.set_ylabel('Flujo [$W m^{-2} nm^{-1}$]')
ax1.set_title(blue_file,size=11,fontweight='bold')

ax2.plot(red_wave_array,red_flux_array,lw=0.4)
ax2.set_xlim(5100,10700)
ax2.set_ylim(-0.05*10**(-12),0.23*10**(-12))
ax2.set_xlabel('Longitud de onda [$\AA$]')
ax2.set_ylabel('Flujo [$W m^{-2} nm^{-1}$]')
ax2.set_title(red_file,size=11,fontweight='bold')

plt.show()

<a class="anchor" id="write1DSpectra"></a>
# Guardar espectros 1D

Si desea guardar los espectros terminados como archivos de texto en lugar de archivos fits, use la receta write1DSpectra como se muestra en las dos celdas siguientes.

Ejecutar la primera celda creará dos archivos. El primero se llamará S20230416S0079_red001_dragons_001.dat y el segundo se llamará S20230416S0079_red001_dragons_002.dat.

Al ejecutar la segunda celda también se crearán dos archivos. El primero se llamará S20230416S0079_blue001_dragons_001.dat y el segundo se llamará S20230416S0079_blue001_dragons_002.dat.

In [ ]:
red_to_1d = ['S20230416S0079_red001_dragons.fits']
reduce_func(red_to_1d,recipename='write1DSpectra')

In [ ]:
blue_to_1d = ['S20230416S0079_blue001_dragons.fits']
reduce_func(blue_to_1d,recipename='write1DSpectra')

<a class="anchor" id="Save_plot"></a>
# Guardar espectros reducidos

Si desea guardar un gráfico en PNG de los espectros rojo y azul reducidos, ejecute la siguiente celda. También tiene la opción de guardar la imagen en un formato diferente, incluidos SVG, eps y PS, reemplazando 'PNG' en la penúltima línea con el formato deseado. Este código guardará los espectros rojo y azul por separado. Un archivo guardado se llamará S20230416S0079_blue001_dragons.png y el otro se llamará S20230416S0079_red001_dragons.png.

In [ ]:
# .- Autor: David Herrera - Junio 2024
# Cree una lista de todos los archivos fits reducidos de DRAGONS en el directorio actual
ls_fits = 'ls -1 *{blue,red}00?*_dragons.fits > dragons_fits.list'
os.system(ls_fits)
# Guardando la lista de DRAGONES reducidos cabe en una lista
fits_list = 'dragons_fits.list'

# Abra la lista de archivos de fits.
with open (fits_list, "r") as files:
    fnames_list = [line.strip() for line in files.readlines()]

# Lea cada nombre de archivo adecuado
for fname in fnames_list:
    # Determinar si es un espectro rojo o azul.
    file = str(fname.strip())
    if '_red' in file:
        band = 'red' 
    else:
        band = 'blue'
    # Abra y lea los datos de cada archivo FITS
    ad = astrodata.open(file)
    flux = ad[0].data
    lam = ad[0].wcs(np.arange(flux.size)).astype(np.float32)
     
    # Convertir λ de nm a Å
    lambda_array = np.array(lam*10)
    flux_array = np.array(flux)
 
    # Definir rangos lambda para cada panel dependiendo de la banda
    if band == 'red':
        lambda_ranges = [(5370, 6330), (6270, 7230), (7170, 8130), (8070, 9030), (8970, 9930)]
    else:
        lambda_ranges = [(3790, 4110), (4090, 4410), (4390, 4710), (4690, 5010), (4990, 5310)]
    # Crea una figura y un conjunto de 5 sub-páneles.
    fig, axs = plt.subplots(len(lambda_ranges), 1, sharex=False, figsize=(10, 8))
 
    # Graficar datos en cada rango
    for i, (lam_min, lam_max) in enumerate(lambda_ranges):
        # Filtrar datos para el rango actual
        mask = (lambda_array >= lam_min) & (lambda_array <= lam_max)
        lambda_filtered = lambda_array[mask]
        flux_filtered = flux_array[mask]
     
        if len(lambda_filtered) > 0 and len(flux_filtered) > 0:
            # Graficar los datos
            axs[i].plot(lambda_filtered, flux_filtered, c=band, lw='.6')
            # Calcule la mediana de flujo del rango de flujo en el rango actual
            flux_median = np.median(flux_filtered)
            # Establecer los límites x
            axs[i].set_xlim(lam_min, lam_max)
            # Establecer los límites y para este panel en particular
            ylim=(-0.25 * flux_median, 2.5 * flux_median)
            axs[i].set_ylim(ylim)
            axs[i].set
            # Ocultar valores de los palitos en y
            # ejes[i].set_yticks([])
            # Manejo de palitos
            axs[i].minorticks_on()
            if band == 'red':
                axs[i].set_xticks(np.arange(lam_min+30,lam_max+30,step=150))
                axs[i].set_xticks(np.arange(lam_min+80,lam_max,step=50), minor = True)
            axs[i].tick_params(axis = 'y', which='major', labelsize = 8)
        else:
            # Manejar el caso en el que no hay puntos de datos en el rango
            axs[i].text(0.5, 0.5, 'No hay datos en este rango', transform=axs[i].transAxes,
                     ha='center', va='center', color=band)
 
        # Opcionalmente, configure la etiqueta Y para cada panel
        # ejes[i].set_yticks()
        # Solo etiqueta y en el tercer panel
        if i == 2: axs[i].set_ylabel('Flujo [$W m^{-2} nm^{-1}$]')
 
    # Establecer la etiqueta del eje x para el gráfico inferior
    axs[-1].set_xlabel('λ(Å)')
    # Establecer título para todo el gráfico.
    fig.suptitle(file)
    # Ajustar el diseño para eliminar espacios entre sub-páneles
    plt.tight_layout()
 
    # Mostrar el gráfico
    # plt.mostrar()
 
    # Guarde el gráfico en un archivo (puede ser png, svg, eps, ps)
    fig.savefig(file.strip('fits') + 'png', dpi='figure', format='png', metadata=None, bbox_inches=None, pad_inches=0.1)
    plt.close()

<a class="anchor" id="IRAF_compatible"></a>
# Hacer compatible con IRAF

Los productos terminados de este notebook cumplen con los estándares fits de DRAGONS, lo que no cumple con lo que espera IRAF. Si desea que los espectros reducidos finales sean compatibles con IRAF, puede utilizar la receta makeIRAFCompatible como se muestra a continuación. (Descomenta antes de ejecutar).

Al ejecutar la primera celda se creará un archivo llamado S20230416S0079_red001_dragons_irafCompatible.fits.

Al ejecutar la segunda celda se creará un archivo llamado S20230416S0079_blue001_dragons_irafCompatible.fits.

In [ ]:
# reduce_iraf = Reduce()
# red_dragons_files = ['S20230416S0079_red001_dragons.fits']
# reducir_iraf.files.extend(red_dragons_files)
# reducir_iraf.recipename = 'makeIRAFCompatible'
# reducir_iraf.runr()

In [ ]:
# reduce_iraf = Reduce()
# blue_dragons_files = ['S20230416S0079_blue001_dragons.fits']
# reducir_iraf.files.extend(blue_dragones_files)
# reducir_iraf.recipename = 'makeIRAFCompatible'
# reducir_iraf.runr()

Este notebook sólo ha utilizado las opciones predeterminadas de DRAGONS. Si desea que todas las exposiciones individuales se reduzcan por separado, puede consultar el comando <a href="https://dragons.readthedocs.io/projects/ghost-drtutorial/en/stable/ex1_ghost_stdonetarget_cmdline.html#alternative-data-products">combineOrders()</a>.

## Opcional: limpiar el directorio de trabajo. (descomentar antes de ejecutar)

In [ ]:
# limpiar_up(save_reduced=0)